In [101]:
from time import sleep
import datetime 
import re
import seaborn as sns
sns.set(style="darkgrid")
import matplotlib.pyplot as plt
import pandas as pd 
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import wiki 
import stock
from scipy.stats import norm
import ppscore as pp
sia = SentimentIntensityAnalyzer()
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import requests
from bs4 import BeautifulSoup
import urllib3
import yfinance as yf
import alpha_vantage
import re
import unicodedata
import test_cleaner
import requests
import numpy as np
import requests
import time

In [2]:
import random

* In this file we try to use the story related to different stocks downloaded from Cloud Media 
* First, let us explore the cleaning method that we only use headlines containing the name of the stock 

In [3]:
start_date = datetime.date(2015,7,1)

In [236]:
senti_ph = '/Users/ball4308/Desktop/MSc BA/BA Report /Sentiment Data Source'
amazon_story = pd.read_csv(senti_ph + '/amazon-all-story-urls-20200815224250.csv')
fb_story = pd.read_csv(senti_ph + '/facebook-all-story-urls-20200815221524.csv')
go_story = pd.read_csv(senti_ph + '/google-all-story-urls-20200815222136.csv')

* First let us use facebook as an example 

In [237]:
amazon_story['publish_date'] = pd.to_datetime(amazon_story['publish_date'])
fb_story['publish_date'] = pd.to_datetime(fb_story['publish_date'])
go_story['publish_date'] = pd.to_datetime(go_story['publish_date'])

In [238]:
amazon_story = amazon_story.sort_values(by = ['publish_date'])
fb_story = fb_story.sort_values(by = ['publish_date'])
go_story = go_story.sort_values(by = ['publish_date'])

In [239]:
fb_story = fb_story.dropna(subset = ['publish_date'])
fb_story = fb_story[fb_story['publish_date'] >= start_date]
fb_story.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17541 entries, 1627 to 18017
Data columns (total 10 columns):
stories_id       17541 non-null int64
publish_date     17541 non-null datetime64[ns]
title            17541 non-null object
url              17541 non-null object
language         17537 non-null object
ap_syndicated    17541 non-null bool
themes           3284 non-null object
media_id         17541 non-null int64
media_name       17541 non-null object
media_url        17541 non-null object
dtypes: bool(1), datetime64[ns](1), int64(2), object(6)
memory usage: 1.4+ MB


In [240]:
print('Available number of days {}'.format(fb_story['publish_date'].apply(lambda x: x.date()).nunique()))
print('Total number of days {}'.format(fb_story['publish_date'].apply(lambda x: x.date()).max() - start_date))

Available number of days 1805
Total number of days 1864 days, 0:00:00


In [241]:
def name_delete(name,x): 
    if name in x: 
        return x 

In [242]:
fb_story['title_FB_only'] = fb_story['title'].apply(lambda x: name_delete('Facebook',x))
fb_story = fb_story.dropna(subset = ['title_FB_only'])
fb_story.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4053 entries, 11775 to 18016
Data columns (total 11 columns):
stories_id       4053 non-null int64
publish_date     4053 non-null datetime64[ns]
title            4053 non-null object
url              4053 non-null object
language         4053 non-null object
ap_syndicated    4053 non-null bool
themes           793 non-null object
media_id         4053 non-null int64
media_name       4053 non-null object
media_url        4053 non-null object
title_FB_only    4053 non-null object
dtypes: bool(1), datetime64[ns](1), int64(2), object(7)
memory usage: 352.3+ KB


In [11]:
print('Available number of days {}'.format(fb_story['publish_date'].apply(lambda x: x.date()).nunique()))
print('Total number of days {}'.format(fb_story['publish_date'].apply(lambda x: x.date()).max() - start_date))

Available number of days 1137
Total number of days 1863 days, 0:00:00


In [12]:
list(fb_story['title'])[0]

'Facebook Will Reap Substitution of TV by Digital Ads, Says Cantor'

In [13]:
fb_story['media_name'].unique()

array(['blogs.barrons.com', 'Reuters', 'Bloomberg', 'Financial Times',
       'Marketwatch'], dtype=object)

In [14]:
fb_story.head()

,stories_id,publish_date,title,url,language,ap_syndicated,themes,media_id,media_name,media_url,title_FB_only
11775,1248303072,2015-07-01 17:33:07,Facebook Will Reap Substitution of TV by Digit...,http://blogs.barrons.com/techtraderdaily/2015/...,en,False,NaN,33038,blogs.barrons.com,http://blogs.barrons.com#spider,Facebook Will Reap Substitution of TV by Digit...
11587,1248308328,2015-07-02 15:12:28,"Yelp Tumbles 11%: Google, Facebook Copying, No...",http://blogs.barrons.com/techtraderdaily/2015/...,en,False,NaN,33038,blogs.barrons.com,http://blogs.barrons.com#spider,"Yelp Tumbles 11%: Google, Facebook Copying, No..."
2483,636875666,2015-07-05 16:50:00,Facebook and birthday cake top this week's cri...,http://feeds.reuters.com/~r/reuters/USVideoMos...,en,False,"crime and criminals, murders and attempted mur...",4442,Reuters,http://www.reuters.com,Facebook and birthday cake top this week's cri...
2570,637006813,2015-07-16 20:43:19,Peru swimmer admits on Facebook to positive do...,http://feeds.reuters.com/~r/reuters/sportsNews...,en,False,NaN,4442,Reuters,http://www.reuters.com,Peru swimmer admits on Facebook to positive do...
11794,1248301115,2015-07-20 09:48:21,"Google, Facebook, Amazon, Apple in ‘Strange Di...",http://blogs.barrons.com/techtraderdaily/2015/...,en,False,NaN,33038,blogs.barrons.com,http://blogs.barrons.com#spider,"Google, Facebook, Amazon, Apple in ‘Strange Di..."


In [15]:
example_url = list(fb_story['url'])[1]
example_url

'http://blogs.barrons.com/techtraderdaily/2015/07/02/yelp-tumbles-11-google-facebook-copying-not-courting-says-axiom/?mod=BOLBlog'

In [16]:
count_word_list = [len(item.split()) for item in list(fb_story['title'])]
sum(count_word_list)/len(count_word_list)

9.927214409079694

In [17]:
req = Request(url=example_url,headers={'user-agent': 'my-app/0.0.1'}) 

In [18]:
webpage = urlopen(req).read()
page_soup_ob = BeautifulSoup(webpage)
#page_soup_ob

In [19]:
article__body = page_soup_ob.find('div','article__body')
print(article__body)

<div class="article__body article-wrap at16-col16 barrons-article-wrap" data-sbid="BL-TB-48260" id="js-article__body" itemprop="articleBody">
<div id="cx-articlecover"></div>
<div class="textresizer">
<span class="textresizer__label">Text size</span>
<button class="textresizer__button textresizer__button--dec" id="js-textresizer--dec"></button>
<button class="textresizer__button textresizer__button--inc" id="js-textresizer--inc"></button>
</div>
<div class="barrons-article-ad-wrapper">
<div class="barrons-article-ad sticky_item" data-track="barrons-article-ad-wrap">
<div class="barrons-main-article-ad-target sticky_target body_ad">
</div>
</div>
</div>
<p>Shares of online local listings and review site <strong>Yelp </strong>(<a class="icon none" href="http://online.barrons.com/public/quotes/main.html?symbol=YELP" target="_blank">YELP</a>) are down $4.61, or 11%, at $37.83, following a story this afternoon by <strong>Bloomberg</strong>'s <strong>Alex Sherman</strong> stating that the co

In [20]:
article_b_t = [item.text for item in article__body.findAll('p')]
article_b_t = ''.join(article_b_t)
article_b_t

'Shares of online local listings and review site\xa0Yelp\xa0(YELP) are down $4.61, or 11%, at $37.83, following a\xa0story this afternoon by\xa0Bloomberg\'s\xa0Alex Sherman stating that the company has decided not to pursue a sale of itself despite\xa0hiring\xa0Goldman Sachs to prospect for suitors, and actually finding "several."The article, citing multiple unnamed sources, states that it\'s up to CEO Jeremy Stoppelman to "change his mind" if the site is to resume talks.First to comment this afternoon from the Street is Victor Anthony of Axiom Capital, who argues Yelp "did not find a buyer," and that actually Google (GOOGL) and Facebook (FB) and Amazon.com (AMZN) are replicating the business.Anthony, who has a Hold rating on Yelp stock, and a $43 price target, writes that there really aren\'t many credible suitor prospects:We argued several times that the list of potential acquirers of Yelp touted by investors such as Google, Facebook, Amazon, and Yahoo! (YHOO) were unlikely to purcha

In [21]:
#test_cleaner.text_pipline(article_b_t)#need to remove the name of the media as well

In [22]:
example_url_2 = list(fb_story['url'])[500]
example_url_2

'https://www.bloomberg.com/news/articles/2016-11-23/facebook-s-quest-to-stop-fake-news-risks-becoming-slippery-slope'

In [23]:
req_2 = Request(url=example_url_2,headers={'user-agent': 'my-app/0.0.1'}) 
webpage_2 = urlopen(req_2).read()
page_soup_ob_2 = BeautifulSoup(webpage_2)
#page_soup_ob_2

In [24]:
article__body_2 = page_soup_ob_2.find('div',"body-copy-v2 fence-body")
article_b_t_2 = [item.text for item in article__body_2.findAll('p')]
article_b_t_2 = ''.join(article_b_t_2)
article_b_t_2

'Last week, a story\xa0claiming that Ford Motor Co. was moving truck production from Mexico to Ohio went viral on Facebook. “The Trump Effect: It’s Happening Already!!” the Facebook user Right Wing News wrote. That story was actually based on a CNN report from 2015, before Donald Trump was even the Republican nominee for president.The post\xa0was neither entirely true nor completely false. It fell into a gray area in the nuanced world of fact-checking, highlighting the thorny challenge of cracking down on fake news. While some articles are obviously fake, like one about the Pope endorsing Trump, many others are misleading, exaggerated or distorted, but contain a kernel of truth. They require judgment calls, and it can be hard to tell where to draw the line, professional fact-checkers say.The Right Wing News post that appeared on Facebook on Nov. 17.“It is a very slippery slope,” said Eugene Kiely, the director of FactCheck.org, a nonprofit that aims to reduce the level of deception and

In [25]:
#test_cleaner.text_pipline(article_b_t_2)

In [26]:
Bloomberg_story = fb_story[fb_story['media_name'] == 'Bloomberg']
Bloomberg_story.head()

,stories_id,publish_date,title,url,language,ap_syndicated,themes,media_id,media_name,media_url,title_FB_only
3272,805032603,2015-07-28 00:00:00,Facebook’s Use of Facial-Recognition Tool Draw...,https://www.bloomberg.com/news/articles/2015-0...,en,False,NaN,40944,Bloomberg,http://bloomberg.com,Facebook’s Use of Facial-Recognition Tool Draw...
5340,912327869,2015-07-30 14:45:48,Facebook Unveils Web-Connected Aircraft With W...,http://www.bloomberg.com/news/articles/2015-07...,en,False,NaN,40944,Bloomberg,http://bloomberg.com,Facebook Unveils Web-Connected Aircraft With W...
11507,912332121,2015-08-14 15:01:58,"Tudor Adds to Facebook, Swaps Investment Grade...",http://www.bloomberg.com/news/articles/2015-08...,en,False,NaN,40944,Bloomberg,http://bloomberg.com,"Tudor Adds to Facebook, Swaps Investment Grade..."
3322,825781338,2015-08-25 00:00:00,Facebook ‘Spam King’ Guilty for Sending 27 Mil...,https://www.bloomberg.com/news/articles/2015-0...,en,False,NaN,40944,Bloomberg,http://bloomberg.com,Facebook ‘Spam King’ Guilty for Sending 27 Mil...
964,537130299,2015-08-27 08:00:00,Germany Tells Facebook: Delete Racist Anti-Ref...,https://www.bloomberg.com/news/articles/2015-0...,en,False,"politics and government, computers and the int...",40944,Bloomberg,http://bloomberg.com,Germany Tells Facebook: Delete Racist Anti-Ref...


In [27]:
Bloomberg_story.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 579 entries, 3272 to 17650
Data columns (total 11 columns):
stories_id       579 non-null int64
publish_date     579 non-null datetime64[ns]
title            579 non-null object
url              579 non-null object
language         579 non-null object
ap_syndicated    579 non-null bool
themes           166 non-null object
media_id         579 non-null int64
media_name       579 non-null object
media_url        579 non-null object
title_FB_only    579 non-null object
dtypes: bool(1), datetime64[ns](1), int64(2), object(7)
memory usage: 50.3+ KB


In [43]:
list(Bloomberg_story['url'])[3]

'https://www.bloomberg.com/news/articles/2015-08-25/facebook-spam-king-guilty-over-27-million-messages-u-s-says'

In [120]:
proxy = pd.read_csv('1990_url.csv')
proxy_ip_list = list(proxy[list(proxy.columns)[0]]) + [proxy.columns[0]]

In [124]:
print(len(proxy_ip_list))

1916


In [125]:
proxy_dic = [{'http':'http://user:pass@' + item +'/'} for item in proxy_ip_list]

In [320]:
Div_dict = {'Bloomberg':'body-copy-v2 fence-body',
            'Barroons':'article__body',
            'R':'StandardArticleBody_body',
            'FT':'article__content-body n-content-body js-article__content-body',
            'MW':'column column--full article__content'}

In [258]:
def get_story(url,clenaed_title,Div_dict,media_name,proxy_ip_list,n): 
    proxy_ip = proxy_ip_list[n]
    proxy_ip = {'http':'http://user:pass@' + proxy_ip +'/'}
    print(url)
    #req = Request(url=url,headers={'user-agent': 'my-app/0.0.1'}) 
    try: 
        time.sleep(3)
        request = requests.get(url, headers={'user-agent': 'my-app/0.0.1'})
        #webpage = urlopen(req).read()
        page_soup = BeautifulSoup(request.text)
        print('The request is blocked  ', page_soup == None)
        #print(page_soup)
        article_body = page_soup.find('div',Div_dict[media_name])
        article = [item.text for item in article_body.findAll('p')]
        print('article extracted')
        return ''.join(article)
    except requests.exceptions.ProxyError:
        print('Proxy Error')
        print('use title for replacement')
        return clenaed_title 
    except requests.exceptions.ConnectionError:
        print('Connection Error')
        print('use title for replacement')
        return clenaed_title 
    except requests.exceptions.HTTPError:
        print('HTTP Error')
        print('use title for replacement')
        return clenaed_title 
    except requests.exceptions.Timeout:
        print('Timeout Error')
        print('use title for replacement')
        return clenaed_title 
    except AttributeError:
        print('Recongised as an Robot')
        print('use title for replacement')
        return clenaed_title 
    except requests.exceptions.TooManyRedirects: 
        print('TooManyRedirects')
        print('use title for replacement')
        return clenaed_title

In [192]:
Bloomberg_story['scraped_article'] = Bloomberg_story['title_FB_only']

In [193]:
Bloomberg_story['BOG'] = Bloomberg_story['scraped_article'].apply(lambda x: test_cleaner.text_pipline(x))

In [194]:
barrons = fb_story[fb_story['media_name'] == 'blogs.barrons.com']

In [195]:
barrons.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 375 entries, 11775 to 17045
Data columns (total 11 columns):
stories_id       375 non-null int64
publish_date     375 non-null datetime64[ns]
title            375 non-null object
url              375 non-null object
language         375 non-null object
ap_syndicated    375 non-null bool
themes           66 non-null object
media_id         375 non-null int64
media_name       375 non-null object
media_url        375 non-null object
title_FB_only    375 non-null object
dtypes: bool(1), datetime64[ns](1), int64(2), object(7)
memory usage: 32.6+ KB


In [196]:
for i, r in barrons.iterrows(): 
    n = random.randint(0,len(proxy_ip_list) -1)
    barrons.at[i, 'scraped_article'] = get_story(r['url'],
                                                 r['title_FB_only'],
                                                 Div_dict,
                                                 'Barroons',
                                                 proxy_ip_list,
                                                 n)


http://blogs.barrons.com/techtraderdaily/2015/07/01/facebook-will-reap-substitution-of-tv-by-digital-ads-says-cantor/?mod=BOLBlog
The request is blocked   False
article extracted
http://blogs.barrons.com/techtraderdaily/2015/07/02/yelp-tumbles-11-google-facebook-copying-not-courting-says-axiom/?mod=BOLBlog
The request is blocked   False
article extracted
http://blogs.barrons.com/techtraderdaily/2015/07/20/google-facebook-amazon-apple-in-strange-divergence-of-nasdaq-say-cnbc/?mod=BOLBlog
The request is blocked   False
article extracted
http://blogs.barrons.com/techtraderdaily/2015/07/21/facebook-street-mulls-what-to-do-with-rocketing-large-cap-internet/?mod=BOLBlog
The request is blocked   False
article extracted
http://online.barrons.com/articles/facebook-stock-primed-to-jump-on-earnings-report-1437595042?mod=rss_barrons_the_striking_price
The request is blocked   False
article extracted
http://blogs.barrons.com/techtraderdaily/2015/07/29/facebook-rises-into-q2-report-as-twitter-yelp-p

In [200]:
barrons.head()

,stories_id,publish_date,title,url,language,ap_syndicated,themes,media_id,media_name,media_url,title_FB_only,scraped_article
11775,1248303072,2015-07-01 17:33:07,Facebook Will Reap Substitution of TV by Digit...,http://blogs.barrons.com/techtraderdaily/2015/...,en,False,NaN,33038,blogs.barrons.com,http://blogs.barrons.com#spider,Facebook Will Reap Substitution of TV by Digit...,Cantor Fitzgerald's Youssef Squali today reite...
11587,1248308328,2015-07-02 15:12:28,"Yelp Tumbles 11%: Google, Facebook Copying, No...",http://blogs.barrons.com/techtraderdaily/2015/...,en,False,NaN,33038,blogs.barrons.com,http://blogs.barrons.com#spider,"Yelp Tumbles 11%: Google, Facebook Copying, No...",Shares of online local listings and review sit...
11794,1248301115,2015-07-20 09:48:21,"Google, Facebook, Amazon, Apple in ‘Strange Di...",http://blogs.barrons.com/techtraderdaily/2015/...,en,False,NaN,33038,blogs.barrons.com,http://blogs.barrons.com#spider,"Google, Facebook, Amazon, Apple in ‘Strange Di...",CNBC’s Bob Pisani had an interesting bit a sho...
11911,1248282901,2015-07-21 11:22:26,Facebook: Street Mulls What To Do With Rocketi...,http://blogs.barrons.com/techtraderdaily/2015/...,en,False,"computers and the internet, finances, stocks a...",33038,blogs.barrons.com,http://blogs.barrons.com#spider,Facebook: Street Mulls What To Do With Rocketi...,"Shares of Facebook (FB) are down 60 cents, or ..."
11547,1248288523,2015-07-23 11:49:20,Facebook Stock Primed to Jump on Earnings Report,http://online.barrons.com/articles/facebook-st...,en,False,NaN,33038,blogs.barrons.com,http://blogs.barrons.com#spider,Facebook Stock Primed to Jump on Earnings Report,Facebook is attracting aggressive upside call ...


In [201]:
barrons['BOG'] = barrons['scraped_article'].apply(lambda x: test_cleaner.text_pipline(x))

In [214]:
barrons.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 375 entries, 11775 to 17045
Data columns (total 13 columns):
stories_id         375 non-null int64
publish_date       375 non-null datetime64[ns]
title              375 non-null object
url                375 non-null object
language           375 non-null object
ap_syndicated      375 non-null bool
themes             66 non-null object
media_id           375 non-null int64
media_name         375 non-null object
media_url          375 non-null object
title_FB_only      375 non-null object
scraped_article    375 non-null object
BOG                375 non-null object
dtypes: bool(1), datetime64[ns](1), int64(2), object(9)
memory usage: 48.5+ KB


In [208]:
fb_story.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4053 entries, 11775 to 18016
Data columns (total 11 columns):
stories_id       4053 non-null int64
publish_date     4053 non-null datetime64[ns]
title            4053 non-null object
url              4053 non-null object
language         4053 non-null object
ap_syndicated    4053 non-null bool
themes           793 non-null object
media_id         4053 non-null int64
media_name       4053 non-null object
media_url        4053 non-null object
title_FB_only    4053 non-null object
dtypes: bool(1), datetime64[ns](1), int64(2), object(7)
memory usage: 512.3+ KB


In [243]:
fb_story['scraped_article'] = ['not_filled'] * fb_story.shape[0]

In [244]:
fb_story.head()

,stories_id,publish_date,title,url,language,ap_syndicated,themes,media_id,media_name,media_url,title_FB_only,scraped_article
11775,1248303072,2015-07-01 17:33:07,Facebook Will Reap Substitution of TV by Digit...,http://blogs.barrons.com/techtraderdaily/2015/...,en,False,NaN,33038,blogs.barrons.com,http://blogs.barrons.com#spider,Facebook Will Reap Substitution of TV by Digit...,not_filled
11587,1248308328,2015-07-02 15:12:28,"Yelp Tumbles 11%: Google, Facebook Copying, No...",http://blogs.barrons.com/techtraderdaily/2015/...,en,False,NaN,33038,blogs.barrons.com,http://blogs.barrons.com#spider,"Yelp Tumbles 11%: Google, Facebook Copying, No...",not_filled
2483,636875666,2015-07-05 16:50:00,Facebook and birthday cake top this week's cri...,http://feeds.reuters.com/~r/reuters/USVideoMos...,en,False,"crime and criminals, murders and attempted mur...",4442,Reuters,http://www.reuters.com,Facebook and birthday cake top this week's cri...,not_filled
2570,637006813,2015-07-16 20:43:19,Peru swimmer admits on Facebook to positive do...,http://feeds.reuters.com/~r/reuters/sportsNews...,en,False,NaN,4442,Reuters,http://www.reuters.com,Peru swimmer admits on Facebook to positive do...,not_filled
11794,1248301115,2015-07-20 09:48:21,"Google, Facebook, Amazon, Apple in ‘Strange Di...",http://blogs.barrons.com/techtraderdaily/2015/...,en,False,NaN,33038,blogs.barrons.com,http://blogs.barrons.com#spider,"Google, Facebook, Amazon, Apple in ‘Strange Di...",not_filled


In [245]:
fb_story.loc[Bloomberg_story.index] = Bloomberg_story 

In [246]:
fb_story[fb_story['media_name'] == 'Bloomberg'].head()

,stories_id,publish_date,title,url,language,ap_syndicated,themes,media_id,media_name,media_url,title_FB_only,scraped_article
3272,805032603,2015-07-28 00:00:00,Facebook’s Use of Facial-Recognition Tool Draw...,https://www.bloomberg.com/news/articles/2015-0...,en,False,NaN,40944,Bloomberg,http://bloomberg.com,Facebook’s Use of Facial-Recognition Tool Draw...,Facebook’s Use of Facial-Recognition Tool Draw...
5340,912327869,2015-07-30 14:45:48,Facebook Unveils Web-Connected Aircraft With W...,http://www.bloomberg.com/news/articles/2015-07...,en,False,NaN,40944,Bloomberg,http://bloomberg.com,Facebook Unveils Web-Connected Aircraft With W...,Facebook Unveils Web-Connected Aircraft With W...
11507,912332121,2015-08-14 15:01:58,"Tudor Adds to Facebook, Swaps Investment Grade...",http://www.bloomberg.com/news/articles/2015-08...,en,False,NaN,40944,Bloomberg,http://bloomberg.com,"Tudor Adds to Facebook, Swaps Investment Grade...","Tudor Adds to Facebook, Swaps Investment Grade..."
3322,825781338,2015-08-25 00:00:00,Facebook ‘Spam King’ Guilty for Sending 27 Mil...,https://www.bloomberg.com/news/articles/2015-0...,en,False,NaN,40944,Bloomberg,http://bloomberg.com,Facebook ‘Spam King’ Guilty for Sending 27 Mil...,Facebook ‘Spam King’ Guilty for Sending 27 Mil...
964,537130299,2015-08-27 08:00:00,Germany Tells Facebook: Delete Racist Anti-Ref...,https://www.bloomberg.com/news/articles/2015-0...,en,False,"politics and government, computers and the int...",40944,Bloomberg,http://bloomberg.com,Germany Tells Facebook: Delete Racist Anti-Ref...,Germany Tells Facebook: Delete Racist Anti-Ref...


In [247]:
fb_story.loc[barrons.index] = barrons
fb_story[fb_story['media_name'] == 'blogs.barrons.com'].head()

,stories_id,publish_date,title,url,language,ap_syndicated,themes,media_id,media_name,media_url,title_FB_only,scraped_article
11775,1248303072,2015-07-01 17:33:07,Facebook Will Reap Substitution of TV by Digit...,http://blogs.barrons.com/techtraderdaily/2015/...,en,False,NaN,33038,blogs.barrons.com,http://blogs.barrons.com#spider,Facebook Will Reap Substitution of TV by Digit...,Cantor Fitzgerald's Youssef Squali today reite...
11587,1248308328,2015-07-02 15:12:28,"Yelp Tumbles 11%: Google, Facebook Copying, No...",http://blogs.barrons.com/techtraderdaily/2015/...,en,False,NaN,33038,blogs.barrons.com,http://blogs.barrons.com#spider,"Yelp Tumbles 11%: Google, Facebook Copying, No...",Shares of online local listings and review sit...
11794,1248301115,2015-07-20 09:48:21,"Google, Facebook, Amazon, Apple in ‘Strange Di...",http://blogs.barrons.com/techtraderdaily/2015/...,en,False,NaN,33038,blogs.barrons.com,http://blogs.barrons.com#spider,"Google, Facebook, Amazon, Apple in ‘Strange Di...",CNBC’s Bob Pisani had an interesting bit a sho...
11911,1248282901,2015-07-21 11:22:26,Facebook: Street Mulls What To Do With Rocketi...,http://blogs.barrons.com/techtraderdaily/2015/...,en,False,"computers and the internet, finances, stocks a...",33038,blogs.barrons.com,http://blogs.barrons.com#spider,Facebook: Street Mulls What To Do With Rocketi...,"Shares of Facebook (FB) are down 60 cents, or ..."
11547,1248288523,2015-07-23 11:49:20,Facebook Stock Primed to Jump on Earnings Report,http://online.barrons.com/articles/facebook-st...,en,False,NaN,33038,blogs.barrons.com,http://blogs.barrons.com#spider,Facebook Stock Primed to Jump on Earnings Report,Facebook is attracting aggressive upside call ...


In [248]:
fb_story.to_csv('bloomberg_barrons_scrping_addedd.csv')

In [249]:
FT = fb_story[fb_story['media_name']  == 'Financial Times']
FT.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 248 entries, 215 to 17088
Data columns (total 12 columns):
stories_id         248 non-null int64
publish_date       248 non-null datetime64[ns]
title              248 non-null object
url                248 non-null object
language           248 non-null object
ap_syndicated      248 non-null bool
themes             32 non-null object
media_id           248 non-null int64
media_name         248 non-null object
media_url          248 non-null object
title_FB_only      248 non-null object
scraped_article    248 non-null object
dtypes: bool(1), datetime64[ns](1), int64(2), object(8)
memory usage: 23.5+ KB


In [252]:
for i, r in FT.iterrows(): 
    n = random.randint(0,len(proxy_ip_list) -1)
    FT.at[i, 'scraped_article'] = get_story(r['url'],
                                                 r['title_FB_only'],
                                                 Div_dict,
                                                 'FT',
                                                 proxy_ip_list,
                                                 n)

http://www.ft.com/cms/s/0/84b73c48-c439-11e5-808f-8231cd71622e.html?ftcamp=published_links%2Frss%2Fworld_uk_politics%2Ffeed%2F%2Fproduct
The request is blocked   False
Recongised as an Robot
use title for replacement
http://www.ft.com/cms/s/0/7c766938-c4e7-11e5-b3b1-7b2481276e45.html?ftcamp=published_links%2Frss%2Fcompanies%2Ffeed%2F%2Fproduct
The request is blocked   False
Recongised as an Robot
use title for replacement
http://www.ft.com/cms/s/0/e89b5ffa-c53d-11e5-b3b1-7b2481276e45.html?ftcamp=published_links%2Frss%2Fhome_europe%2Ffeed%2F%2Fproduct
The request is blocked   False
Recongised as an Robot
use title for replacement
http://www.ft.com/cms/s/0/e89b5ffa-c53d-11e5-b3b1-7b2481276e45.html?ftcamp=published_links%2Frss%2Fcompanies%2Ffeed%2F%2Fproduct
The request is blocked   False
Recongised as an Robot
use title for replacement
http://www.ft.com/cms/s/3/165acad0-c54c-11e5-808f-8231cd71622e.html?ftcamp=published_links%2Frss%2Fcompanies%2Ffeed%2F%2Fproduct
The request is blocked   

KeyboardInterrupt: 

In [253]:
MW = fb_story[fb_story['media_name']  == 'Marketwatch']
MW.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 663 entries, 3371 to 17646
Data columns (total 12 columns):
stories_id         663 non-null int64
publish_date       663 non-null datetime64[ns]
title              663 non-null object
url                663 non-null object
language           663 non-null object
ap_syndicated      663 non-null bool
themes             120 non-null object
media_id           663 non-null int64
media_name         663 non-null object
media_url          663 non-null object
title_FB_only      663 non-null object
scraped_article    663 non-null object
dtypes: bool(1), datetime64[ns](1), int64(2), object(8)
memory usage: 62.8+ KB


In [254]:
list(MW['url'])[0]

'http://feeds.marketwatch.com/~r/marketwatch/marketpulse/~3/S-qV4JxoDrs/story.aspx'

In [269]:
for i, r in MW.iterrows(): 
    n = random.randint(0,len(proxy_ip_list) -1)
    MW.at[i, 'scraped_article'] = get_story(r['url'],
                                                 r['title_FB_only'],
                                                 Div_dict,
                                                 'MW',
                                                 proxy_ip_list,
                                                 n)

http://feeds.marketwatch.com/~r/marketwatch/marketpulse/~3/S-qV4JxoDrs/story.aspx
The request is blocked   False
article extracted
http://feeds.marketwatch.com/~r/marketwatch/bulletins/~3/YCGh-fx-HKQ/bulletinredir.asp
The request is blocked   False
article extracted
http://feeds.marketwatch.com/~r/marketwatch/marketpulse/~3/SYOVBruSWww/story.aspx
The request is blocked   False
article extracted
http://feeds.marketwatch.com/~r/marketwatch/internet/~3/CVDBrc7IBhE/story.asp
The request is blocked   False
article extracted
http://feeds.marketwatch.com/~r/marketwatch/bulletins/~3/N-JjRXt1M9g/bulletinredir.asp
The request is blocked   False
Recongised as an Robot
use title for replacement
http://feeds.marketwatch.com/~r/marketwatch/financial/~3/RZZqnAYImns/story.asp
The request is blocked   False
article extracted
http://feeds.marketwatch.com/~r/marketwatch/pf/~3/Ohv1nfuHijg/story.asp
The request is blocked   False
article extracted
http://feeds.marketwatch.com/~r/marketwatch/internet/~3/7dq

In [272]:
MW['scraped_article'][5804]

'BRUSSELS—The European Union’s top justice official is set to quiz her U.S. counterparts over privacy issues on a visit to Washington this week made more timely by fallout over Facebook Inc.’s handling of personal data.The planned U.S. visit by EU Justice Commissioner Věra Jourová comes after\n        FB, \n        +0.45%\n      announced that a data-analysis firm with ties to the 2016 Trump campaign wrongfully held users’ personal data for years despite saying it had deleted those records.The news sent the social-media firm’s shares tumbling\xa0on Monday\xa0and sparked outcry by European officials, including Jourová and European Parliament President Antonio Tajani, who pledged a full parliamentary investigation.Read:Cambridge Analytica caught on hidden camera pitching bribes and sex workersJourová said she would discuss the developments in meetings planned with U.S. officials, which include Attorney General Jeff Sessions and Commerce Secretary Wilbur Ross. Her team has also requested 

In [275]:
len(test_cleaner.text_pipline(MW['scraped_article'][5804]))

178

In [303]:
fb_story.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4053 entries, 11775 to 18016
Data columns (total 12 columns):
stories_id         4053 non-null int64
publish_date       4053 non-null datetime64[ns]
title              4053 non-null object
url                4053 non-null object
language           4053 non-null object
ap_syndicated      4053 non-null bool
themes             793 non-null object
media_id           4053 non-null int64
media_name         4053 non-null object
media_url          4053 non-null object
title_FB_only      4053 non-null object
scraped_article    4053 non-null object
dtypes: bool(1), datetime64[ns](1), int64(2), object(8)
memory usage: 543.9+ KB


In [304]:
fb_story.loc[MW.index] = MW

In [307]:
fb_story[fb_story['media_name']  == 'Marketwatch'].head()

,stories_id,publish_date,title,url,language,ap_syndicated,themes,media_id,media_name,media_url,title_FB_only,scraped_article
3371,827822656,2018-03-19 15:04:28,Facebook hires digital-audit firm to investiga...,http://feeds.marketwatch.com/~r/marketwatch/ma...,en,False,NaN,35170,Marketwatch,https://www.marketwatch.com/,Facebook hires digital-audit firm to investiga...,Facebook Inc. said Monday that it has hired th...
3365,827815937,2018-03-19 15:16:50,Facebook's 7% drop is the worst daily decline ...,http://feeds.marketwatch.com/~r/marketwatch/bu...,en,False,NaN,35170,Marketwatch,https://www.marketwatch.com/,Facebook's 7% drop is the worst daily decline ...,"Shares of Facebook Inc.\n FB, \n ..."
3368,827822626,2018-03-19 18:31:48,Facebook security chief to exit: report,http://feeds.marketwatch.com/~r/marketwatch/ma...,en,False,NaN,35170,Marketwatch,https://www.marketwatch.com/,Facebook security chief to exit: report,"Facebook Inc.\n FB, \n +0.45%\n ..."
5804,827817959,2018-03-20 07:15:06,The Wall Street Journal: Facebook backlash to ...,http://feeds.marketwatch.com/~r/marketwatch/in...,en,False,NaN,35170,Marketwatch,https://www.marketwatch.com/,The Wall Street Journal: Facebook backlash to ...,BRUSSELS—The European Union’s top justice offi...
3363,827815876,2018-03-20 08:59:54,Facebook's stock-price decline continues,http://feeds.marketwatch.com/~r/marketwatch/bu...,en,False,NaN,35170,Marketwatch,https://www.marketwatch.com/,Facebook's stock-price decline continues,Facebook's stock-price decline continues


In [322]:
def get_story_with_proxy_ip(url,clenaed_title,Div_dict,media_name,proxy_ip_list,n): 
    """
    input: 
    url = url of the website
    clenaed_title = cleaned title of the article, if the web extraction of the 
                    full article is failed, return the cleaned title 
    Div_dict = a dictionary pre-defined to store the name of the media and the 
                    <div> format of HTML file 
    media_name = name of media: from Bloomberg, Barroons, Retures, FT and MarketWatch
    proxy_ip_list = a purchased proxy list used to increase chance of successful 
                    web scraping 
    n = randomly generated integer for use in a loop, for the proxy ip indexing 
    """
    proxy_ip = proxy_ip_list[n]
    proxy_ip = {'http':'http://user:pass@' + proxy_ip +'/'}

    try: 
        time.sleep(3)
        request = requests.get(url, 
                               proxies = proxy_ip,
                               headers={'user-agent': 'my-app/0.0.1'})
        page_soup = BeautifulSoup(request.text)
        print('The request is blocked  ', page_soup == None)
        article_body = page_soup.find('div',Div_dict[media_name])
        article = [item.text for item in article_body.findAll('p')]
        print('article extracted')
        return ''.join(article)
    except requests.exceptions.ProxyError:
        print('Proxy Error')
        print('use title for replacement')
        return clenaed_title 
    except requests.exceptions.ConnectionError:
        print('Connection Error')
        print('use title for replacement')
        return clenaed_title 
    except requests.exceptions.HTTPError:
        print('HTTP Error')
        print('use title for replacement')
        return clenaed_title 
    except requests.exceptions.Timeout:
        print('Timeout Error')
        print('use title for replacement')
        return clenaed_title 
    except AttributeError:
        print('Recongised as an Robot')
        print('use title for replacement')
        return clenaed_title 
    except requests.exceptions.TooManyRedirects: 
        print('TooManyRedirects')
        print('use title for replacement')
        return clenaed_title

In [323]:
FT.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 248 entries, 215 to 17088
Data columns (total 12 columns):
stories_id         248 non-null int64
publish_date       248 non-null datetime64[ns]
title              248 non-null object
url                248 non-null object
language           248 non-null object
ap_syndicated      248 non-null bool
themes             32 non-null object
media_id           248 non-null int64
media_name         248 non-null object
media_url          248 non-null object
title_FB_only      248 non-null object
scraped_article    248 non-null object
dtypes: bool(1), datetime64[ns](1), int64(2), object(8)
memory usage: 33.5+ KB


#### FT scrapping is impossible, always get blocked 

In [314]:
R = fb_story[fb_story['media_name']  == 'Reuters']
R.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2188 entries, 2483 to 18016
Data columns (total 12 columns):
stories_id         2188 non-null int64
publish_date       2188 non-null datetime64[ns]
title              2188 non-null object
url                2188 non-null object
language           2188 non-null object
ap_syndicated      2188 non-null bool
themes             409 non-null object
media_id           2188 non-null int64
media_name         2188 non-null object
media_url          2188 non-null object
title_FB_only      2188 non-null object
scraped_article    2188 non-null object
dtypes: bool(1), datetime64[ns](1), int64(2), object(8)
memory usage: 207.3+ KB


In [319]:
R['url'][10272]

'https://www.reuters.com/article/us-israel-palestinians-socialmedia/israel-says-facebook-youtube-videos-encouraging-palestinian-attacks-idUSKCN0S21K020151008'

### most of the url's genrate connection errors, so will use title for replacement 

In [328]:
R['scraped_article'] = R['title_FB_only']
FT['scraped_article'] = FT['title_FB_only']

In [329]:
fb_story.loc[R.index] = R
fb_story.loc[FT.index] = FT

In [330]:
fb_story[fb_story['scraped_article'] == 'not_filled']

,stories_id,publish_date,title,url,language,ap_syndicated,themes,media_id,media_name,media_url,title_FB_only,scraped_article


In [332]:
fb_story['BOG'] = fb_story['scraped_article'].apply(lambda x: test_cleaner.text_pipline(x))

In [336]:
fb_story.to_csv('fb_story_withBOG.csv')

In [ ]:
def BOG_cleaner(bog): 
    
